In [2]:
import pandas as pd
import json, os
from datetime import datetime
import numpy as np

In [3]:
providers = {
    'aws': 'aws-amazon-web-services-reports',
    'gcp': 'google-cloud-reports',
    'azure': 'windows-azure-reports'
}

In [8]:
df = pd.DataFrame(columns={'event_id': [], 'timestamp': [], 'count' : [], 'provider': []})


for provider, reports in providers.items():
    path = os.getcwd() + "/down-detector/" + reports
    provider_html_pages = os.listdir(path)
    event_id = 1
    for provider_page in provider_html_pages:
        file = open(path + "/" + provider_page, "r")
        json_obj = json.load(file)
        counts_obj = json_obj["counts"]
        
        for count in counts_obj:
            timestamp = datetime.strptime(count['x'], '%Y-%m-%dT%H:%M:%S%z').timestamp()
            nr_report = count['y']
            df = df.append({'timestamp': timestamp, 'event_id': event_id, 'provider': provider, 'count': nr_report}, ignore_index=True)
        
        event_id += 1

In [10]:
df = df.sort_values('timestamp').reset_index(drop=True, inplace=False)
df.to_csv('down_detector_outage_count.csv', index=False)

In [11]:
df_reasons = pd.DataFrame(columns={'report_id': [], 'event_id': [], 'timestamp': [], 'count' : [], 'provider': [], 'cause': [], 'percent': []})

report_id = 0

for provider, reports in providers.items():
    path = os.getcwd() + "/down-detector/" + reports
    provider_html_pages = os.listdir(path)
    event_id = 1
    for provider_page in provider_html_pages:
        file = open(path + "/" + provider_page, "r")
        json_obj = json.load(file)
        counts_obj = json_obj["counts"]
        reasons_obj = json_obj.get("reasons")
        for count in counts_obj:
            timestamp = datetime.strptime(count['x'], '%Y-%m-%dT%H:%M:%S%z').timestamp()
            nr_report = count['y']
            if reasons_obj != None:
                for reason in reasons_obj:
                    cause = list(reason.keys())[0]
                    percent = float(list(reason.values())[0].split("%")[0]) / 100 
                    df_reasons = df_reasons.append({'report_id': report_id, 'event_id': event_id, 'timestamp': timestamp, 'count': nr_report, 'provider': provider, 'cause': cause, 'percent': percent}, ignore_index=True)
                    
                report_id += 1
                
            else:
                df_reasons = df_reasons.append({'report_id': report_id, 'event_id': event_id, 'timestamp': timestamp, 'count': nr_report, 'provider': provider, 'cause': np.NaN, 'percent': np.NaN}, ignore_index=True)
                report_id += 1
        
        event_id += 1

In [12]:
df_reasons.sort_values('timestamp').reset_index(drop=True, inplace=True)
df_reasons.to_csv('down_detector_reported_problem_count.csv', index=False)